# Create Dataset from .zip files and calc normalization statistics

Step_1: create standard dataset to be re-used, either by creating folders or as hdf5 object. 

Step_2: calculate mean + std for train and test images

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from common import shuffle_zip
BASE = "./datasets/YNet_data_02/re_crop/"

## Run scripts to generate ready_data

In [3]:
ds_name = BASE+'yeast_v4.4'
shuffle_zip(ds_name+"/original_zips",ds_name+'/shuffled_zips/',ds_name)

working on [mfb1KO] class
deleted 39 non square images.
{False}
0 251
length of train: [216]; length of test: [35]
Files moved to:./datasets/YNet_data_02/re_crop/yeast_v4.4/shuffled_zips/
train created
train/mfb1KO created
test created
test/mfb1KO created
working on [mmr1KO] class
deleted 61 non square images.
{False}
0 347
length of train: [312]; length of test: [35]
Files moved to:./datasets/YNet_data_02/re_crop/yeast_v4.4/shuffled_zips/
working on [mmr1KO_mfb1KO] class
deleted 57 non square images.
{False}
0 319
length of train: [284]; length of test: [35]
Files moved to:./datasets/YNet_data_02/re_crop/yeast_v4.4/shuffled_zips/
working on [WT] class
deleted 37 non square images.
{False}
0 223
length of train: [188]; length of test: [35]
Files moved to:./datasets/YNet_data_02/re_crop/yeast_v4.4/shuffled_zips/
